In [9]:
# Part1
# ============================================================
# Bayesian Network: Burglary–Earthquake–Alarm–Calls
# ============================================================

# ------------------------------------------------------------
# PRIOR PROBABILITIES
# ------------------------------------------------------------
P_B = 0.001      # Burglary
P_E = 0.002      # Earthquake

# ------------------------------------------------------------
# CPT: P(A | B, E)
# ------------------------------------------------------------
P_A_given = {
    (True,  True): 0.95,
    (True,  False): 0.94,
    (False, True): 0.29,
    (False, False): 0.001
}

# Helper function to compute P(A)
def marginal_P_A():
    return (
        P_A_given[(True, True)]   * P_B * P_E +
        P_A_given[(True, False)]  * P_B * (1 - P_E) +
        P_A_given[(False, True)]  * (1 - P_B) * P_E +
        P_A_given[(False, False)] * (1 - P_B) * (1 - P_E)
    )

# ============================================================
# QUESTION 1: Independence without evidence
# ============================================================
print("QUESTION 1 — Are B and E independent without evidence?\n")

P_B_and_E = P_B * P_E  # because BN topology says they are independent a priori

print(f"P(B) = {P_B}")
print(f"P(E) = {P_E}")
print(f"P(B ∧ E) = P(B) * P(E) = {P_B_and_E:.8f}")
print("\nTopological reasoning: B → A ← E is a collider.")
print("Collider unobserved → path blocked → B and E are d-separated → Independent.\n")


# ============================================================
# QUESTION 2: Conditional independence given Alarm=True
# ============================================================
print("\nQUESTION 2 — Are B and E independent given Alarm = True?\n")

P_A = marginal_P_A()
print(f"P(A) = {P_A:.8f}")

# ------------------------------------------------------------
# Compute P(B | A)
# ------------------------------------------------------------
numerator_B_given_A = (
    P_B * P_E * P_A_given[(True, True)] +
    P_B * (1 - P_E) * P_A_given[(True, False)]
)
P_B_given_A = numerator_B_given_A / P_A
print(f"P(B | A) = {P_B_given_A:.6f}")

# ------------------------------------------------------------
# Compute P(B | A, E=True)
# ------------------------------------------------------------
num = P_B * P_A_given[(True, True)]
den = (
    P_B * P_A_given[(True, True)] +
    (1 - P_B) * P_A_given[(False, True)]
)
P_B_given_A_E_true = num / den
print(f"P(B | A, E=True) = {P_B_given_A_E_true:.6f}")

# ------------------------------------------------------------
# Compute P(B | A, E=False)
# ------------------------------------------------------------
num = P_B * P_A_given[(True, False)]
den = (
    P_B * P_A_given[(True, False)] +
    (1 - P_B) * P_A_given[(False, False)]
)
P_B_given_A_E_false = num / den
print(f"P(B | A, E=False) = {P_B_given_A_E_false:.6f}")

# ------------------------------------------------------------
# Conclusion
# ------------------------------------------------------------
print("\nCONCLUSION:")
print("If P(B | A) ≠ P(B | A, E), then B and E are NOT conditionally independent given A.")
print(f"P(B | A)          = {P_B_given_A:.6f}")
print(f"P(B | A, E=True)  = {P_B_given_A_E_true:.6f}")
print(f"P(B | A, E=False) = {P_B_given_A_E_false:.6f}")
print("\nTherefore: Burglary and Earthquake become dependent given Alarm=True (explaining away).")


QUESTION 1 — Are B and E independent without evidence?

P(B) = 0.001
P(E) = 0.002
P(B ∧ E) = P(B) * P(E) = 0.00000200

Topological reasoning: B → A ← E is a collider.
Collider unobserved → path blocked → B and E are d-separated → Independent.


QUESTION 2 — Are B and E independent given Alarm = True?

P(A) = 0.00251644
P(B | A) = 0.373551
P(B | A, E=True) = 0.003268
P(B | A, E=False) = 0.484786

CONCLUSION:
If P(B | A) ≠ P(B | A, E), then B and E are NOT conditionally independent given A.
P(B | A)          = 0.373551
P(B | A, E=True)  = 0.003268
P(B | A, E=False) = 0.484786

Therefore: Burglary and Earthquake become dependent given Alarm=True (explaining away).


In [8]:
# Part2
from pgmpy.models import DiscreteBayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

# -------------------------------
# Define the Bayesian Network structure
# -------------------------------
model = DiscreteBayesianNetwork([
    ("Attendance", "Participation"),
    ("Attendance", "Homework"),
    ("IndependentWork", "Homework"),
    ("OneOnOne", "IndependentWork"),
    ("OneOnOne", "Performance"),
    ("Participation", "Performance"),
    ("Homework", "Performance"),
    ("IndependentWork", "Performance")
])

# -------------------------------
# Define Conditional Probability Distributions
# -------------------------------

# Attendance (prior)
cpd_attendance = TabularCPD(
    variable="Attendance",
    variable_card=2,
    values=[[0.75], [0.25]],  # True=0.75, False=0.25
    state_names={"Attendance": ["True", "False"]}
)

# Participation given Attendance
cpd_participation = TabularCPD(
    variable="Participation",
    variable_card=2,
    values=[[0.70, 0.25], [0.30, 0.75]],
    evidence=["Attendance"],
    evidence_card=[2],
    state_names={"Participation": ["True", "False"], "Attendance": ["True", "False"]}
)

# Homework given Attendance and IndependentWork
cpd_homework = TabularCPD(
    variable="Homework",
    variable_card=2,
    values=[
        [0.90, 0.75, 0.65, 0.30],  # HW=True
        [0.10, 0.25, 0.35, 0.70]   # HW=False
    ],
    evidence=["Attendance", "IndependentWork"],
    evidence_card=[2, 2],
    state_names={
        "Homework": ["True", "False"],
        "Attendance": ["True", "False"],
        "IndependentWork": ["True", "False"]
    }
)

# IndependentWork given OneOnOne
cpd_independent = TabularCPD(
    variable="IndependentWork",
    variable_card=2,
    values=[[0.70, 0.40], [0.30, 0.60]],
    evidence=["OneOnOne"],
    evidence_card=[2],
    state_names={"IndependentWork": ["True", "False"], "OneOnOne": ["True", "False"]}
)

# OneOnOne (prior)
cpd_oneonone = TabularCPD(
    variable="OneOnOne",
    variable_card=2,
    values=[[0.30], [0.70]],
    state_names={"OneOnOne": ["True", "False"]}
)

# Performance given Participation, Homework, IndependentWork, OneOnOne
# Simplified noisy-OR style: more "True" parents → higher chance of Perf=True
cpd_performance = TabularCPD(
    variable="Performance",
    variable_card=2,
    values=[
        # Perf=True probabilities
        [0.95, 0.92, 0.82, 0.80, 0.70, 0.60, 0.40, 0.15,
         0.85, 0.75, 0.65, 0.50, 0.55, 0.45, 0.30, 0.10],
        # Perf=False probabilities
        [0.05, 0.08, 0.18, 0.20, 0.30, 0.40, 0.60, 0.85,
         0.15, 0.25, 0.35, 0.50, 0.45, 0.55, 0.70, 0.90]
    ],
    evidence=["Participation", "Homework", "IndependentWork", "OneOnOne"],
    evidence_card=[2, 2, 2, 2],
    state_names={
        "Performance": ["True", "False"],
        "Participation": ["True", "False"],
        "Homework": ["True", "False"],
        "IndependentWork": ["True", "False"],
        "OneOnOne": ["True", "False"]
    }
)

# -------------------------------
# Add CPDs to the model
# -------------------------------
model.add_cpds(
    cpd_attendance,
    cpd_participation,
    cpd_homework,
    cpd_independent,
    cpd_oneonone,
    cpd_performance
)

# Check model consistency
assert model.check_model()

# -------------------------------
# Inference
# -------------------------------
infer = VariableElimination(model)

# Example query: Probability of good performance given Attendance=True and Homework=True
q = infer.query(variables=["Performance"], evidence={"Attendance": "True", "Homework": "True"})
print(q)


+--------------------+--------------------+
| Performance        |   phi(Performance) |
+====================+====================+
| Performance(True)  |             0.8117 |
+--------------------+--------------------+
| Performance(False) |             0.1883 |
+--------------------+--------------------+


In [6]:
# Part3
import math
import random
from collections import defaultdict

random.seed(2025)

# -------------------------------
# Setup: 8 athletes and synthetic data
# -------------------------------
athletes = [f"Athlete_{i+1}" for i in range(8)]

# Generate true latent abilities (unknown to the model; used to create data)
true_abilities = {name: random.gauss(10.50, 0.25) for name in athletes}

# Observational noise per round (realistic: tighter in later rounds)
sigma_R1, sigma_R2, sigma_R3 = 0.09, 0.08, 0.07
sigma_final = 0.06  # final race variability

# Generate observed times from latent ability (this is the "data")
observations = {}
for name in athletes:
    theta = true_abilities[name]
    t1 = random.gauss(theta, sigma_R1)
    t2 = random.gauss(theta, sigma_R2)
    t3 = random.gauss(theta, sigma_R3)
    observations[name] = {"R1": round(t1, 3), "R2": round(t2, 3), "R3": round(t3, 3)}

# -------------------------------
# Bayesian inference (Normal–Normal conjugacy)
# -------------------------------
mu0 = 10.50   # prior mean ability (s)
sigma0 = 0.30 # prior std dev (s)

def posterior_for_athlete(times, sigmas, mu0, sigma0):
    """
    Given a list of observed times and their std devs, return posterior (mu, sigma).
    """
    tau0 = 1.0 / (sigma0 ** 2)
    tau_obs_sum = sum(1.0 / (s ** 2) for s in sigmas)
    sigma_post_sq = 1.0 / (tau0 + tau_obs_sum)
    weighted_sum = (mu0 * tau0) + sum(t / (s ** 2) for t, s in zip(times, sigmas))
    mu_post = sigma_post_sq * weighted_sum
    sigma_post = math.sqrt(sigma_post_sq)
    return mu_post, sigma_post

# Compute posteriors for all athletes using R1, R2, R3
posteriors = {}
for name in athletes:
    times = [observations[name]["R1"], observations[name]["R2"], observations[name]["R3"]]
    sigmas = [sigma_R1, sigma_R2, sigma_R3]
    mu_post, sigma_post = posterior_for_athlete(times, sigmas, mu0, sigma0)
    posteriors[name] = (mu_post, sigma_post)

# -------------------------------
# Posterior predictive for the final (simulate winners)
# -------------------------------
def sample_final_time(mu_post, sigma_post, sigma_final):
    """
    Sample from posterior predictive: Normal(mu_post, sigma_post^2 + sigma_final^2).
    """
    std_pred = math.sqrt(sigma_post ** 2 + sigma_final ** 2)
    return random.gauss(mu_post, std_pred)

def estimate_win_probabilities(posteriors, n_sims=20000):
    win_counts = defaultdict(int)
    rank_counts = {name: defaultdict(int) for name in athletes}

    for _ in range(n_sims):
        final_times = []
        for name in athletes:
            mu_post, sigma_post = posteriors[name]
            t_final = sample_final_time(mu_post, sigma_post, sigma_final)
            final_times.append((name, t_final))
        # Rank by time ascending (fastest is winner)
        final_times.sort(key=lambda x: x[1])
        # Tally winner and ranks
        winner = final_times[0][0]
        win_counts[winner] += 1
        for rank, (name, _) in enumerate(final_times, start=1):
            rank_counts[name][rank] += 1

    win_probs = {name: win_counts[name] / n_sims for name in athletes}
    rank_probs = {
        name: {r: rank_counts[name][r] / n_sims for r in sorted(rank_counts[name].keys())}
        for name in athletes
    }
    return win_probs, rank_probs

win_probs, rank_probs = estimate_win_probabilities(posteriors, n_sims=20000)

# -------------------------------
# Reporting
# -------------------------------
print("=== Observed times (data) ===")
for name in athletes:
    r = observations[name]
    print(f"{name}: R1={r['R1']}s  R2={r['R2']}s  R3={r['R3']}s")

print("\n=== Posterior over ability (per athlete) ===")
for name in athletes:
    mu_post, sigma_post = posteriors[name]
    print(f"{name}: mu={mu_post:.3f}s, sigma={sigma_post:.3f}s")

print("\n=== Estimated probability of winning the final ===")
for name, p in sorted(win_probs.items(), key=lambda x: -x[1]):
    print(f"{name}: {p:.3f}")

print("\n=== Probability of finishing at each rank (1..8) ===")
for name in athletes:
    ranks = rank_probs[name]
    probs_str = ", ".join(f"{r}:{ranks.get(r,0):.3f}" for r in range(1, 9))
    print(f"{name}: {probs_str}")


=== Observed times (data) ===
Athlete_1: R1=10.141s  R2=10.177s  R3=10.185s
Athlete_2: R1=10.119s  R2=10.393s  R3=10.385s
Athlete_3: R1=10.453s  R2=10.364s  R3=10.356s
Athlete_4: R1=10.463s  R2=10.568s  R3=10.624s
Athlete_5: R1=11.385s  R2=11.151s  R3=11.071s
Athlete_6: R1=10.347s  R2=10.394s  R3=10.624s
Athlete_7: R1=10.323s  R2=10.36s  R3=10.43s
Athlete_8: R1=10.526s  R2=10.598s  R3=10.486s

=== Posterior over ability (per athlete) ===
Athlete_1: mu=10.179s, sigma=0.045s
Athlete_2: mu=10.324s, sigma=0.045s
Athlete_3: mu=10.386s, sigma=0.045s
Athlete_4: mu=10.563s, sigma=0.045s
Athlete_5: mu=11.162s, sigma=0.045s
Athlete_6: mu=10.480s, sigma=0.045s
Athlete_7: mu=10.383s, sigma=0.045s
Athlete_8: mu=10.532s, sigma=0.045s

=== Estimated probability of winning the final ===
Athlete_1: 0.886
Athlete_2: 0.079
Athlete_7: 0.018
Athlete_3: 0.016
Athlete_6: 0.001
Athlete_8: 0.000
Athlete_4: 0.000
Athlete_5: 0.000

=== Probability of finishing at each rank (1..8) ===
Athlete_1: 1:0.886, 2:0.094,